<H1>Linear Tree Algorithm experiment</H1>

1.) Collect, clean exemplary dataset for regression problem suited for linear tree model.<br>
2.) Develop single tree algorithm with variables for depth, omitted features as input (others to be added later) and feature importance and classification solution as output<br>
3.) Define simple Gradient Boosting algorithm<br>
4.) Bring full algorithm together<br>
5.) Run tests with exemplary dataset<br>

In [1]:
import pandas as pd
import numpy as np
import plotly as pl
import plotly.express as px

import itertools

pd.set_option('display.max_rows', 500)

In [2]:
#data cleaning and pre-preparation completed in EDA module
df=pd.read_csv('data/ames_housing_price_data_prepared_v01.csv')
df['SalePriceLog']=np.log10(df['SalePrice'])
df=df.drop(['Unnamed: 0','SalePrice'],axis=1)

#price=df['SalePrice']
#price_log = np.log10(price)
#df.head(10).T

In [3]:
#Reduction of feature space to 15 features and dependent variable to reduce computational requirements.
columnlist=['GrLivArea', 'LotArea', 'OverallQual', 'BSMT_LowQual',
       'house_age_years', 'GarageCars', 'FullBath', 'HalfBath',
       'BsmtExposure_ord', 'SaleTypeNew', 'PorchSF', 'BSMT_HighQual',
       'Fireplaces', 'Pool', 'BedroomAbvGr', 'SalePriceLog']
df=df[columnlist]

<H1>Initial Benchmark: XGBoost Regression model</H1><br>
    Model has been optimized using a cross-validated Grid Search and tested on 30% of data separately afterwards
    

In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(df.drop('SalePriceLog',axis=1),df['SalePriceLog'],test_size=0.33)

model = xgb.XGBRegressor()

params={
        'min_child_weight': [1, 5, 7],
        'gamma': [0,0.1, 0.5, 1],
        'subsample': [0.5,1.0],
        'colsample_bytree': [0.8,1],
        'max_depth': [2,3,4,6,10]
}

grid=GridSearchCV(model, params,cv=3)
grid.fit(X_train,y_train)
print(f'Score: ',grid.score(X_test,y_test))
print('Params: ',grid.best_params_)


Score:  0.8860828156828164
Params:  {'colsample_bytree': 1, 'gamma': 0, 'max_depth': 2, 'min_child_weight': 5, 'subsample': 1.0}


In [5]:
#Model performance at 88.61% as benchmark

<H1>Tree Booster model with Ridge Regression as loss function

'''
Overall setup:
- Function that runs trees sequentially, predicting the previous tress residuals, and after each tree updates a 'Residuals' column with tree performance and learning rate
- Function that calculates each tree, by iterating through all nodes, and for each node all columns and splitpoints within these colums. Tree is then always split by node that offers highest MSE-Reduction potential. Uses DP to save nodes that were already visited.
- Function that calculates Ridge Regression coefficients for each node-split option, and MSE for the Ridge predictor.
- Function that predicts Residuals and MSE for each tree after tree has been finalized
- Function that finally allows to predict new datapoints.
- Function to calculate R2 score for final treebooster model.
'''

In [6]:
class TreeBooster():
    
    def __init__(self, n_nodes=16, n_iterations=10, learning_rate=0.1, max_depth=6, min_elements=1,alpha=1):
        '''
        Initialize class and save key parameters for booster. Initialize dictionaries/ lists to store 
        split_conditions during training, and linear regression coefficienys for each exit node.
        '''
        self.n_nodes=n_nodes
        self.n_iterations=n_iterations
        self.learning_rate=learning_rate
        self.max_depth=max_depth
        self.min_elements=min_elements
        self.alpha=alpha
        
        self.split_conditions={}
        self.tree_coefs={}
        self.tree_depth={}
        self.columns=[]
        
        self.initial_mean=0
        
        #feature_importance_dict={}
        
    def ridge_coefs(self,df):
        '''
        #based on dependent and predictor columns, run RidgeRegression fit and score for each       
        Input: dataframe with relevant rows
        Output: List of coefficients beta_0 in first position, beta_1 to beta_n following
        '''

        onerow=np.ones(df.shape[0]).reshape(df.shape[0],1)
        
        #Split data into predictor and dependent vaiables
        X_vals=np.c_[onerow,np.matrix(df[self.columns])]
        Y_vals=np.matrix(df['Residuals'])
        Identity=np.eye(X_vals.shape[1])
        Identity[0,0]=0
        
        #Ridge solution for coefficients: 𝛽=(𝑋𝑇𝑋+𝜆𝐼)−1𝑋𝑇𝑦.
        coefs=(X_vals.T.dot(X_vals)+self.alpha*Identity).I.dot(X_vals.T).dot(Y_vals.T)
        return coefs
    
    def check_error(self,df):
        '''
        For a subset dataframe, calculate the error (currently only setup for MSE) of the potential leaf node.
        Input: dataframe with relevant rows
        Output: Error as float (MSE)
        '''
        
        #Collect Ridge regression coefficients from function ridge_coefs()
        coefs=self.ridge_coefs(df)
        error=sum(np.square(np.matrix(df['Residuals']).T-(np.matrix(df[self.columns])*coefs[1:]+coefs[0])))/df.shape[0]
        return error

    def treebuilder(self, df, tree_num=0):
        '''
        Create a decision tree, by iterating each node, testing all splits for each feature and each 
        split_value within the feature, then split the tree based on the feature generating highest MSE-reduction
        Input: Dataframe with a column called 'Residuals' as dependent variable, and all columns as defined
        in class variable self.columns as Predictor variables.
        Output: Updated split_conditions dicitionary, providing for each node the next split conditions, and
        subnote_ids. Can be used to run a full prediction based on the tree splitting criteria.
        '''
        
        #Set each observation to tree_node 0 (this will be adjusted once we start splitting the tree)
        df['subtree_id']=0
        
        max_subtree_id=0
        
        #Set tmp_dict to save visited nodes (subtree_ids) and their loss-reduction performance.
        tmp_dict={}
        
        #Initialize dictionaries to save split conditions, and coefficients for leaf regression for each node.
        self.split_conditions[tree_num]={}
        self.tree_coefs[tree_num]={}
        self.tree_depth[tree_num]={}
        
        self.tree_depth[tree_num][0]=0

        k=0
        
        #Execute for specified number of nodes
        while k<self.n_nodes:

            subtree_ids=df['subtree_id'].unique()
            
            for subtree_id in subtree_ids: #iterate through each current subtree
                
                #stop, if max_depth for node already reached
                if self.tree_depth[tree_num][subtree_id]<self.max_depth:
                    
                    currentError=self.check_error(df[df['subtree_id']==subtree_id])
                    
                    for column in self.columns:#Iterate each feature

                        if (subtree_id,column) not in tmp_dict:#Check if we already checked this node previously
                            
                            #Define split values as the average between each two observations in a sorted range
                            split_values=list(df[df['subtree_id']==subtree_id][column].unique())
                            split_values.sort()
                            split_val=[]
                            for i in range(1, len(split_values)):
                                split_val.append(split_values[i-1]+(split_values[i]-split_values[i-1])/2)

                            maxError=0
                            total_shape=df[(df['subtree_id']==subtree_id)].shape[0]
                            
                            for i in split_val:#iterate through each split
                                
                                bottom_shape=df[(df['subtree_id']==subtree_id) & (df[column]<i)].shape[0]
                                top_shape=df[(df['subtree_id']==subtree_id) & (df[column]>=i)].shape[0]
                                
                                if bottom_shape>=self.min_elements and top_shape>=self.min_elements:
                                    
                                    #Check MSE for each resuliting sub_node for given split and calculate weighted average
                                    newError=(self.check_error(df[(df['subtree_id']==subtree_id) & (df[column]<i)])*bottom_shape + self.check_error(df[(df['subtree_id']==subtree_id) & (df[column]>=i)])*top_shape)/total_shape

                                    deltaError=currentError-newError

                                    #if DeltaError is larger than maxError for this node/feature combination, save it to consider it for future split
                                    if deltaError>maxError:
                                        tmp_dict[(subtree_id,column)]=(i,deltaError)
                                        maxError=deltaError
            
            #Check if we have tested any splits (could be no, if e.g. all nodes at max_depth)
            if tmp_dict:
                
                #Collect best split from tmp_dict, and set subtree_id in dataframe to two new values for newly defined split
                best_subtree_id, best_column=max(tmp_dict, key=lambda x: tmp_dict.get(x)[1])
                best_split_pos=tmp_dict[(best_subtree_id, best_column)][0]
                max_subtree_id+=1
                df.loc[(df['subtree_id']==best_subtree_id) & (df[best_column]<best_split_pos),'subtree_id']=max_subtree_id
                self.tree_depth[tree_num][max_subtree_id]=self.tree_depth[tree_num][best_subtree_id]+1
                max_subtree_id+=1
                df.loc[(df['subtree_id']==best_subtree_id)&(df[best_column]>=best_split_pos),'subtree_id']=max_subtree_id
                self.tree_depth[tree_num][max_subtree_id]=self.tree_depth[tree_num][best_subtree_id]+1
                
                #Delete all keys from tmp dict that are now invalid since we split the node they checked.
                for i,j in list(tmp_dict.keys()):
                    if i==best_subtree_id:
                        del tmp_dict[(i,j)]   
                        
                #Save splitting conditions        
                self.split_conditions[tree_num][best_subtree_id]=(best_column,best_split_pos,max_subtree_id-1,max_subtree_id)
            
            k+=1
            
            #print(df['subtree_id'].value_counts())

        #calculate ridge regression coefficients for each subtree, use coefficients when predicting datapoints
        subtree_ids=df['subtree_id'].unique()
        for subtree_id in subtree_ids: #each subtree
            coefs=self.ridge_coefs(df[df['subtree_id']==subtree_id])
            self.tree_coefs[tree_num][subtree_id]=coefs

        return df
    
    
    def tree_predictor(self, df, tree_num=0):
        '''
        Predict the dependent variable for a single tree based on the previously defined tree-logic.
        Input: dataframe with all training data and split_conditions dictionary outlining how to split the tree.
        Output: Dataframe with attached column 'Predict' containing the predictions
        '''
        df['Predict']=0
        
        for i in df.index:
            current_subtree_id=0
            while current_subtree_id not in self.tree_coefs[tree_num]:
                column,split_pos,id_left,id_right = self.split_conditions[tree_num][current_subtree_id]
                if df[column][i]<split_pos:
                    current_subtree_id=id_left
                else:
                    current_subtree_id=id_right
                    
            coefs=self.tree_coefs[tree_num][current_subtree_id]
            prediction=float(np.matrix(df.loc[i,self.columns])*coefs[1:]+coefs[0])
            df.loc[i,'Predict']=prediction

        return df


    def iteration_builder(self, df):
        '''
        Booster logic: Initialize fitting procedure by setting the base prediction as the mean of all valus, then
        start predicting and iteratively updating the residuals tree by tree.
        Input: Dataframe with Predictor column named 'SalesPriceLog' (should be generalized later) as the last 
        column in the dataframe, and all previous columns as predictor features.
        Output: Split_conditions dictionary, putlinign for each tree the split conditions
        '''
        df2=df.copy()
        
        #Define all predictor columns and save into class
        self.columns=list(df2.columns)
        self.columns.pop()
        
        #Initialze model by setting prediction to mean of all dependent variable observations
        df2['Base_prediction']=df2['SalePriceLog'].mean()
        self.initial_mean=df2['SalePriceLog'].mean()
        df2['Residuals']=df2['SalePriceLog']-df2['Base_prediction']
        
        res_mean=df2['Residuals'].map(lambda x: x**2).mean()
        print(f'Starting; residual mean at {res_mean}')
        
        #Build trees and update residuals based on each tree
        for x in range(self.n_iterations):
            df2=self.treebuilder(df2,tree_num=x)
            df2=self.tree_predictor(df2,tree_num=x)
            df2['Base_prediction']=df2['Base_prediction']+df2['Predict']*self.learning_rate
            df2['Residuals']=df2['SalePriceLog']-df2['Base_prediction']
            df2=df2.drop(['Predict'],axis=1)
            
            res_mean=df2['Residuals'].map(lambda x: x**2).mean()
            print(f'Iteration {x} complete, residual mean at {res_mean}')
        return df2
    
    def iteration_predictor(self, df):
        '''
        Predict dependent variable for new observations by sequentially creating each tree, and calculating 
        new residual based on the new tree.
        Input: Dataframe with each observation
        Output: Dataframe with extra column containing Predicted values for each observation.
        '''
        df2=df.copy()
        df2=df2.reset_index(drop=False)

        df2['Predict_fin']=self.initial_mean
        for x in range(self.n_iterations):
            df2=self.tree_predictor(df2,tree_num=x)
            df2['Predict_fin']=df2['Predict_fin']+self.learning_rate*df2['Predict']
            
        df2['Predict']=df2['Predict_fin']
        df2=df2.drop('Predict_fin',axis=1)
        
        df2=df2.set_index('index',drop=True)
        df2.index.name=None

        return df2
    
    def tree_scorer(self,df):
        '''
        Calculate R2 score for any dataframe.
        Input: Dataframe with two columns: Depedent variable and predicted values per observation
        Output: R2 score
        '''
        mean1=sum(df.iloc[:,0])/df.shape[0]
        TSS=sum(df.iloc[:,0].map(lambda x: (x-mean1)**2))

        RSS=sum((df.iloc[:,1]-df.iloc[:,0])**2)

        return 1-RSS/TSS
        

In [7]:
def GridSearchCV(df):
    '''
    Basic GridSearch function: For now, alywas uses #3 splits, each based on random bagging of 67% 
    of observations in train for Cross-Validation.
    Currently set to always use same GridSeatch Options and model (can be restructured as inputs later)
    Input: Dataframe with train data (last column should be dependent variable, with name 'SalePriceLog')
    Output: Average cross-validation test score and parameters for best parameter combination
    '''
    
    
    scores={}
    
    options={'n_nodes': [8,10],
             'n_iterations': [25],
             'learning_rate': [0.1],
             'max_depth': [3,4],
             'min_elements': [10,20], 
             'alpha': [0.1,20]
            }
    
    iterables=list(itertools.product(options['n_nodes'],options['n_iterations'],options['learning_rate'],options['max_depth'],options['min_elements'],options['alpha']))
    
    for n_nodes,n_iterations,learning_rate,max_depth,min_elements,alpha in iterables:
        
        tmpscore=[]
        
        for _ in range(3):
    
            train_indices=list(np.random.choice(df.index, size=int(df.shape[0]*0.667),replace=False))
            test_indices=[x if x not in train_indices else -1 for x in df.index]
            d_train=df.loc[df.index.isin(train_indices)]
            d_test=df.loc[df.index.isin(test_indices)]
    
    
            treemodel=TreeBooster(n_nodes=n_nodes, n_iterations=n_iterations, learning_rate=learning_rate, max_depth=max_depth, min_elements=min_elements,alpha=alpha)
            d_train= treemodel.iteration_builder(d_train)
    
            d_test=d_test.reset_index(drop=False)
            d_test2=treemodel.iteration_predictor(d_test)
            score=treemodel.tree_scorer(d_test2[['SalePriceLog','Predict']])
            
            tmpscore.append(score)
            
        scores[sum(tmpscore)/len(tmpscore)]=(n_nodes,n_iterations,learning_rate,max_depth,min_elements,alpha)
        
        print(scores[max(scores.keys())], max(scores.keys()))
    
    return scores[max(scores.keys())]


In [8]:
train_indices=list(np.random.choice(df.index, size=int(df.shape[0]*0.7),replace=False))   #int(df.shape[0]*0.7)
test_indices=[x if x not in train_indices else -1 for x in df.index]
df_train=df.loc[df.index.isin(train_indices)]
df_test=df.loc[df.index.isin(test_indices)]

In [9]:
#n_nodes,n_iterations,learning_rate,max_depth,min_elements,alpha = GridSearchCV(df_train)
#print(n_nodes,n_iterations,learning_rate,max_depth,min_elements,alpha)

In [10]:
n_nodes,n_iterations,learning_rate,max_depth,min_elements,alpha = (6, 25, 0.1, 4, 40, 0.001)

In [ ]:
treemodel=TreeBooster(n_nodes=n_nodes, n_iterations=n_iterations, learning_rate=learning_rate, max_depth=max_depth, min_elements=min_elements,alpha=alpha)
df_train= treemodel.iteration_builder(df_train)

df_test=df_test.reset_index(drop=False)
df_test2=treemodel.iteration_predictor(df_test)
treemodel.tree_scorer(df_test2[['SalePriceLog','Predict']])


Starting; residual mean at 0.026840117957575325
Iteration 0 complete, residual mean at 0.02212114925739094
Iteration 1 complete, residual mean at 0.01828673982825497
Iteration 2 complete, residual mean at 0.015211318434035068
Iteration 3 complete, residual mean at 0.012670886238429789
Iteration 4 complete, residual mean at 0.010615896673640674
Iteration 5 complete, residual mean at 0.008950710433264224
Iteration 6 complete, residual mean at 0.007580763600133983
Iteration 7 complete, residual mean at 0.006472235242499397
Iteration 8 complete, residual mean at 0.00556409778842203
Iteration 9 complete, residual mean at 0.004830306399768823
Iteration 10 complete, residual mean at 0.004217527538948213
Iteration 11 complete, residual mean at 0.0037161693143886827
Iteration 12 complete, residual mean at 0.003305164890852075
Iteration 13 complete, residual mean at 0.002967973426617146
Iteration 14 complete, residual mean at 0.0026964481080186302
Iteration 15 complete, residual mean at 0.002469

In [ ]:
#without Ridge: 0.9024810993000432
#XGBoost fully optimized: 0.90325821780744
#Basic Linear Regression (not penalized): 0.8974280280636848

#with Ridge, 400 obs, not optimized: 0.8073027768102685
#Ridge, all obs, not optimized: 0.8651592190813793
#Ridge 0.9091286769287041 #(n_iter=6, n_iterations=25, learning_rate=0.1, max_depth=4, min_elements=30, alpha=0.1)
#0.9108590610477793 at (n_iter=6, n_iterations=25, learning_rate=0.1, max_depth=4, min_elements=40, alpha=0.001)
#0.9033206921634097 (8, 25, 0.1, 4, 20, 0.1) 